## Usando arquivo .env para controlar variaveis de ambiente
Para evitar exposição da chave `OPENAI_API_KEY` optei por utilizar arquivo `.env` com a informação da chave.

Para seguir o mesmo método basta criar um arquivo `.env` no mesmo diretório do arquivo `code-review_rag.ipynb`.
A importação da chave será feita através da célula abaixo que faz a instalação de um biblioteca para carregar
os valores do arquivo `.env`.

In [17]:
%pip install python-dotenv
import dotenv
%load_ext dotenv
%dotenv

Note: you may need to restart the kernel to use updated packages.
The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


### Importando Libs

Neste vídeo, é demonstrado como construir um RAG para código utilizando o Jupyter Notebook. São importadas diversas bibliotecas, como langchain, chroma, openai, chat-openai, entre outras. São utilizados métodos para carregar documentos, separar linguagens de programação e criar correntes de perguntas e respostas. Também é mostrado como instalar as bibliotecas necessárias, como langchain-chroma e langchain-openai. Além disso, é feita a importação das bibliotecas os e git para trabalhar com o sistema operacional e baixar o código de exemplo diretamento do Github.

In [1]:
from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.parsers import LanguageParser
from langchain_text_splitters import Language, RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains.question_answering import load_qa_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

import os
from git import Repo

### Carregando o código

Neste trecho, é demonstrado como criar uma variável para definir o caminho de um repositório e baixar um código específico para análise. É feito o carregamento de arquivos Python de um diretório específico, excluindo aqueles que não seguem o padrão UTF-8. O processo envolve o uso de um loader genérico, parser de linguagem Python e definição de um threshold. Ao final, é exibido o número de documentos carregados para análise.

In [2]:
repo_path = "./test_repo"

In [3]:
# clonar o repositório - código do langchain
repo = Repo.clone_from("https://github.com/langchain-ai/langchain", to_path=repo_path)

In [ ]:
# carregar o repositório
loader = GenericLoader.from_filesystem(
    repo_path + "/libs/core/langchain_core/",
    glob="**/*", # carregar todos os arquivos
    suffixes=[".py"], # filtro para arquivos python
    exclude=["**/non-utf-8-encoding.py"], # arquivos com encondings diferentes
    parser=LanguageParser(language=Language.PYTHON, parser_threshold=500) # 
)

documents = loader.load()
len(documents)

475

### Separando documentos em Chunks

Ao carregar os documentos, aplicamos o recurso text splitter para separá-los em chunks. A escolha do tamanho do chunk, como 2.000 neste caso, é crucial e varia de acordo com o tipo de documento, como código ou PDF. Na arquitetura RAG, é essencial encontrar o tamanho ideal do chunk para otimizar o processo. Testes são necessários para determinar o tamanho mais adequado, considerando a base de dados. A execução do método resultou em 1.278 chunks a partir de 410 documentos.

In [ ]:
# separação de pedaços de documentos (chunks)
python_spliter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, chunk_size=2000, chunk_overlap = 200
)

texts = python_spliter.split_documents(documents)
len(texts)

1417

### Método retriever

Abordamos a criação do Embedding Module usando a chave de API da OpenAI, a construção do banco vetorial Chroma e a configuração do Retriever para buscar documentos relevantes. Exploramos a inicialização do ChromaDB com OpenAI Embeddings, a definição do Search Type como MMR e a especificação de Keywords como 8.

In [ ]:
# criar um banco de dados vetorial com os textos
db = Chroma.from_documents(texts, OpenAIEmbeddings(disallowed_special=()))

# criar um retriever para pesquisar os textos - objeto buscador
retriever = db.as_retriever(
    search_type = "mmr", # teste com similaridade diferente
    search_kwargs = {"k": 8}, # quantidade de textos buscados
)

### Construindo o prompt e a chain

Neste trecho, é explicado o uso do LLM ChatOpenAI, especialmente o modelo GPT-3.5 Turbo, com a definição de um prompt utilizando o método ChatPromptTemplate. O prompt é criado com informações do System, User e Assistant, para melhorar a resposta do modelo. É mencionada a criação de uma DocumentChain e RetrievalChain para compilar documentos e realizar a recuperação de textos. O objetivo é utilizar essas ferramentas para responder a consultas de revisão de código de forma mais eficiente.

In [ ]:
# associar um LLM a uma variável para gerar a resposta
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo", # modelo llm
    max_tokens=200, # máximo de tokens
)

In [ ]:
# criar um chain para gerar a resposta (prompt)
prompt = ChatPromptTemplate.from_messages([
    (
        "system", # sistema
        "Você é um revisor de código experiente. Forneça informações detalhadas sobre a revisão do código e sugestões de melhorias baseadas no contexto fornecido abaixo: \n\n{context}", # context
    ),
    ("user", "{input}"), # query
])

# compilar os docs recebidos para receber o llm e o prompt
document_chain = create_stuff_documents_chain(llm, prompt)

# cadeia de recuperação dos textos 
retrieval_chain = create_retrieval_chain(retriever, document_chain)

### Response e Recapitulando

Foi destacada a importância da documentação, clareza e organização do código, além de sugestões de melhorias, como completar a documentação, incluir exemplos práticos, testes unitários e refatoração do código. Também foi mencionada a evolução da arquitetura e a resolução de problemas futuros.

In [ ]:
# buscador de textos passando um input específico
response = retrieval_chain.invoke({"input": "Você pode revisar e sugerir melhorias para o código de RunnableBinding"})
response["answer"]

'O código de `RunnableBinding` parece estar bem estruturado e organizado. No entanto, há algumas sugestões de melhorias e observações que podem ser consideradas:\n\n1. **Documentação Adequada**: Adicionar documentação detalhada e explicativa para cada método e classe, incluindo descrições claras, exemplos de uso e possíveis cenários de aplicação.\n\n2. **Tipagem Forte**: Certifique-se de adicionar anotações de tipo explícitas para parâmetros de método e valores de retorno, para facilitar a compreensão do código e garantir a consistência.\n\n3. **Tratamento de Exceções**: Considere adicionar tratamento de exceções apropriado em métodos como `backward`, `update` e outros, para lidar com possíveis erros de forma mais robusta.\n\n4. **Melhorias nos Métodos**: \n   - No'